<a href="https://colab.research.google.com/github/andrewmlu/super-stable-rep/blob/main/CS242_Project_Scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import requests
import json

url = "https://stablediffusionapi.com/api/v3/text2img"

payload = json.dumps({
  "key": "",
  "prompt": "golden labrador in a field of sunflowers",
  "negative_prompt": "grass",
  "width": "512",
  "height": "512",
  "samples": "4",
  "num_inference_steps": "31",
  "seed": None,
  "guidance_scale": 7.5,
  "safety_checker": "yes",
  "multi_lingual": "no",
  "panorama": "no",
  "self_attention": "no",
  "upscale": "no",
  "embeddings_model": None,
  "webhook": None,
  "track_id": None
})

headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)




{"status":"success","tip":"Get 20x faster image generation using enterprise plan. Click here : https:\/\/stablediffusionapi.com\/enterprise","generationTime":2.391197681427002,"id":56150219,"output":["https:\/\/cdn2.stablediffusionapi.com\/generations\/5ca5feb9-2b02-4d67-b34b-4c677a5fb6f0-0.png","https:\/\/cdn2.stablediffusionapi.com\/generations\/5ca5feb9-2b02-4d67-b34b-4c677a5fb6f0-1.png","https:\/\/cdn2.stablediffusionapi.com\/generations\/5ca5feb9-2b02-4d67-b34b-4c677a5fb6f0-2.png","https:\/\/cdn2.stablediffusionapi.com\/generations\/5ca5feb9-2b02-4d67-b34b-4c677a5fb6f0-3.png"],"proxy_links":"","nsfw_content_detected":"","meta":{"H":512,"W":512,"enable_attention_slicing":"true","file_prefix":"5ca5feb9-2b02-4d67-b34b-4c677a5fb6f0","guidance_scale":7.5,"instant_response":"no","model":"runwayml\/stable-diffusion-v1-5","n_samples":4,"negative_prompt":"grass","outdir":"out","prompt":"golden labrador in a field of sunflowers","revision":"fp16","safetychecker":"yes","seed":3292235381,"ste

In [38]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("drive/My Drive/!!!Drive/CS242-SD-imgs")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
img_srcs = json.loads(response.text)['output']

In [41]:
import requests
from PIL import Image

In [66]:
def save_img(url):

  # This statement requests the resource at
  # the given link, extracts its contents
  # and saves it in a variable
  data = requests.get(url).content
  print(data)

  # Opening a new file named img with extension .png
  # This file would store the data of the image file
  f = open('./'+url[48:],'wb')

  # Storing the image data inside the data variable to the file
  f.write(data)
  f.close()

  # Opening the saved image and displaying it
  img = Image.open('./'+url[48:])
  img.show()


In [67]:
save_img(img_srcs[1])

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x00\x00\x00\x02\x00\x08\x02\x00\x00\x00{\x1aC\xad\x00\x073\x14IDATx\xdad\xbd[\x92$\xdbn,\x06\x07VdU\xf7\xe6&%\x9a\xe6\xa5+\x8d@\xe2\'\xa5ai"w\n\xfa\x93\xe9K&\xb3k2\xfd\xe8r?\xaa23\x16\xe0\xfa\xc0#\xa2\xc9\xc3s\x8c{wWe\xc6c-,\xc0\xe1\xee\xc0\xff\xf6\x9f\xff\xf7\x90\xc3\xb9U\x90\xff\x11P\x02\x80P(\x02\x85\x90\x94\xfa\x0fE \x00D\xc8\xa8?\xa2\xa0\xfeL\x84B\x88\x90\xe8\xbf\x030\xbf,\x02\x81\x90\xacO\'I\x02\xa2\nP\x82B\x06\x14\ne~(#?_\x00\x05\xae\xafS\x88@HA\xfe\xb50\xbf\x11\x02\x11\x8a\x08\xc0\x10\xd5\xfck\xccE\x8a\x10\x022?\x12\x92\x17+\x02A\x90\x10P("\n\xe4\x8f\x90!\x80*\xear\xea\x0f\t\x00"\x91\xbf\xd9\x1f\x9dOGHU\rw\x81@\xc0\xbc\x14\x11\xa8F\x90B(\x84""\x11\xa1"\xa2\xa0@XW\x9e7~=\xf2\xfeR\xe4\x93\xab/d\xdei}k_\xad)"\xea\xb6\xf3\xf1\xaa\xe6\xbd\x80\xf5.\xf2\x87\xaf\x87\x17\x11R\x8fB\xeai)\xe6mS\xf2\xae\xf3\x19\x0b\xa4\xbf\x92u\xa9\n\x01@\xf6\xe5\xa2\x1f\x1d\x10QO\x89"*B\xe6/\xd5\xabd\xe4M\xf7]\xe7\xaa\x92~\xdb\xf5U\xec\'P\xb7Y\xcb\x89\xb7\